In [1]:
import random
import gymnasium as gym
from pprint import pprint
from catanatron import Color,RandomPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer, DevCardRandomPlayer, DoNothingRandomPlayer,CityRandomPlayer,SettlementRandomPlayer,LongestRoadRandomPlayer
from catanatron_gym.rewards import reward_function
from catanatron_gym.features import create_sample_vector, create_sample
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json

from catanatron.players.search import VictoryPointPlayer
from catanatron_experimental.machine_learning.players.value import ValueFunctionPlayer
from catanatron_gym.envs.catanatron_env import from_action_space
from catanatron.json import GameEncoder
import gymnasium as gym
import numpy as np

from stable_baselines3 import PPO
from catanatron_gym.features import create_sample_vector, get_feature_ordering,create_sample
from catanatron_gym.envs.catanatron_env import from_action_space
# FEATURES = get_feature_ordering
from pprint import pprint
import random
import pandas as pd

from catanatron_experimental.machine_learning.players.minimax import AlphaBetaPlayer

from selenium import webdriver
from catanatron_server.utils import open_link,ensure_link
import webbrowser
import time
import os

In [2]:
def render(game,ep,step):
    ep_dir = f"./games/episode_{ep}/states"
    geckodriver_path = "/snap/bin/geckodriver"
    driver_service = webdriver.FirefoxService(executable_path=geckodriver_path)

    driver = webdriver.Firefox(service=driver_service)
    link = ensure_link(game)
    driver.get(link)
    time.sleep(1)
    # driver.get_screenshot_as_png('image.png')
    driver.save_screenshot(f'./games/episode_{ep}/states/step_{step}.png')
    # print(driver.get_screenshot_as_png()) 
    driver.close()
    # TODO: save using screenshot as png then convert to pil

In [5]:
def run_switch_agent(env_config, agent_path="./logs/Catan_Switch_Exp/exp_019/latest_model_10000000_steps"):
    # "./logs/Catan_Switch_Exp/exp_019/latest_model_10000000_steps"
    best_path = "../logs/Catan_Switch_Exp/exp_014/best_model.zip"
    latest_path = "../logs/Catan_Switch_Exp/exp_004/latest_model_1000000_steps.zip"
    features = None
    try:
        env = gym.make("catanatron_gym:catanatron-switch-v1", config=env_config)
        print("Made env using config")
    except:
        env = gym.make("catanatron_gym:catanatron-switch-v1")
        print('Made env without config')

    observation, info = env.reset()
    model = PPO.load(path=agent_path, env=env)

    for ep in range(20):
        ep_state_dir = f"./games/episode_{ep}/states"
        os.makedirs(ep_state_dir, exist_ok=True)
        episode_data = []  # Store data for current episode
        episode_info = []  # Store info for current episode

        for step in range(1000):
            action = model.predict(observation=observation, deterministic=True)
            record = create_sample(env.game, env.p0.color)
            features = sorted(record.keys())  # Get list of feature names in same order as observation
            episode_data.append([ep, step, observation.tolist(), action[0]])
            # print(observation)
            observation, reward, terminated, truncated, info = env.step(action[0])
            # render(env.game,ep=ep,step=step)
            done = terminated or truncated
            # print(env.p0.color)
            if done:
                # env.unwrapped.game.state()
                game_json = GameEncoder().default(env.game)

                episode_info.append(info) 
                save_episode_data(ep, episode_data, episode_info, features,env)
                episode_info.append(info)  # Save info at the end of each episode
                # print(env.p0.color)
                # render(env.game,ep=ep,step=step)
                observation, info = env.reset()
                break

        # Save the episode data and info
        # save_episode_data(ep, episode_data, episode_info, features,env)

    env.close()

def save_episode_data(ep, data, info, features,env):
    # Create directory for the episode
    ep_dir = f"./games/episode_{ep}"
    os.makedirs(ep_dir, exist_ok=True)
    ep_state_dir = f"./games/episode_{ep}/states"
    os.makedirs(ep_state_dir, exist_ok=True)

    # Convert data to DataFrame and save as CSV
    df = pd.DataFrame(data, columns=["ep","step",'obs', 'action'])
    print(df.dtypes)
    df.to_csv(f"{ep_dir}/episode_{ep}_data.csv", index=False)

    # Save info as JSON
    with open(f"{ep_dir}/episode_{ep}_info.json", 'w') as f:
        json.dump(info, f, indent=4)

    game_json = GameEncoder().default(env.game)
    # pprint(game_json)
    print(info)
    # my_json= json.loads(game_json)
    with open(f"{ep_dir}/episode_{ep}_game.json",'w') as f:
        f.write(json.dumps(game_json, cls=GameEncoder,indent=4))
        # json.dump(game_json, f, indent=4,cls=GameEncoder)
    with open(f"{ep_dir}/episode_{ep}_game.txt",'w') as f:
        f.write(str(game_json))
        # json.dump(game_json, f, indent=4,cls=GameEncoder)
    # # Optionally save feature list
    # with open(f"{ep_dir}/episode_{ep}_features.json", 'w') as f:
    #     json.dump(features, f, indent=4)


In [6]:
config={
        "enemies": [VictoryPointPlayer(Color.RED)],
        # "enemies": [ValueFunctionPlayer(Color.RED)],
        # "enemies": [AlphaBetaPlayer(Color.RED,depth=1)],
        "map_type":"TOURNAMENT"}
run_switch_agent(config)

running switch env
{'enemies': [VictoryPointPlayer:RED], 'map_type': 'TOURNAMENT'}
Made env using config
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/alex/anaconda3/envs/catan/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


ep         int64
step       int64
obs       object
action    object
dtype: object
[{'game_stats': {'actions_count': 0, 'num_turns': 236, 'p0_winning': True}, 'p0_stats': {'p0_vp': 10, 'p0_actual_vp': 10, 'p0_has_road': True, 'p0_has_army': True, 'p0_longest_road_length': 7, 'p0_cities': 1, 'p0_settlements': 4}, 'p1_stats': {'p1_vp': 2, 'p1_actual_vp': 2, 'p1_has_road': False, 'p1_has_army': False, 'p1_longest_road_length': 1, 'p1_cities': 0, 'p1_settlements': 2}, 'valid_actions': {'valid_actions': [289, 276, 273, 275, 274]}}]


/home/alex/anaconda3/envs/catan/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catan/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


ep         int64
step       int64
obs       object
action    object
dtype: object
[{'game_stats': {'actions_count': 0, 'num_turns': 164, 'p0_winning': True}, 'p0_stats': {'p0_vp': 9, 'p0_actual_vp': 10, 'p0_has_road': True, 'p0_has_army': False, 'p0_longest_road_length': 12, 'p0_cities': 1, 'p0_settlements': 5}, 'p1_stats': {'p1_vp': 2, 'p1_actual_vp': 2, 'p1_has_road': False, 'p1_has_army': False, 'p1_longest_road_length': 8, 'p1_cities': 0, 'p1_settlements': 2}, 'valid_actions': {'valid_actions': [289]}}]


...

/home/alex/anaconda3/envs/catan/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catan/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


ep         int64
step       int64
obs       object
action    object
dtype: object
[{'game_stats': {'actions_count': 0, 'num_turns': 192, 'p0_winning': True}, 'p0_stats': {'p0_vp': 10, 'p0_actual_vp': 10, 'p0_has_road': True, 'p0_has_army': True, 'p0_longest_road_length': 5, 'p0_cities': 2, 'p0_settlements': 2}, 'p1_stats': {'p1_vp': 3, 'p1_actual_vp': 3, 'p1_has_road': False, 'p1_has_army': False, 'p1_longest_road_length': 4, 'p1_cities': 0, 'p1_settlements': 3}, 'valid_actions': {'valid_actions': [289, 39, 40, 23, 57, 61, 60, 81, 32, 85, 52, 67, 69]}}]


/home/alex/anaconda3/envs/catan/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catan/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


ep         int64
step       int64
obs       object
action    object
dtype: object
[{'game_stats': {'actions_count': 0, 'num_turns': 485, 'p0_winning': False}, 'p0_stats': {'p0_vp': 2, 'p0_actual_vp': 2, 'p0_has_road': False, 'p0_has_army': False, 'p0_longest_road_length': 2, 'p0_cities': 0, 'p0_settlements': 2}, 'p1_stats': {'p1_vp': 8, 'p1_actual_vp': 11, 'p1_has_road': True, 'p1_has_army': True, 'p1_longest_road_length': 12, 'p1_cities': 0, 'p1_settlements': 4}, 'valid_actions': {'valid_actions': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19]}}]


/home/alex/anaconda3/envs/catan/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
